In [1]:
import tensorflow
import tensorflow.keras as keras
import tensorflow.keras.applications as ka
# from tensorflow.keras.keras_preprocessing.image import ImageDataGenerator
import numpy as np
import sys
# import sklearn.model_selection as skl
from sklearn.model_selection import train_test_split

import matplotlib
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"]=1,1

In [2]:
base_model = ka.ResNet50(
#                         include_top=False,
                        weights='imagenet',#,
#                         classes=2,
                        input_shape = (224, 224, 3)
                        )

for layer in base_model.layers:
    layer.trainable = False

# base_model_add_layers = keras.Model(inputs = base_model.input, outputs = base_model.output)
# out_1 = keras.layers.GlobalAvgPool2D()(base_model.output)
# out_1 = keras.layers.GlobalAveragePooling2D()(base_model.output)
# x = keras.layers.Flatten()(base_model.layers[-2].output)
out_2 = keras.layers.Dense(1600, activation='relu')(base_model.layers[-2].output)
out_1 = keras.layers.Dense(1000, activation='softmax')(out_2)

model = keras.Model(inputs = base_model.input, outputs = out_1)#, base_model.output, base_model.layers[-2].output])
model_extra = keras.Model(inputs = model.input, outputs = [model.layers[-1].output, model.layers[-2].output, model.layers[-3].output])
# opt = keras.optimizers.SGD(lr = 0.0001)
opt = keras.optimizers.Adam(learning_rate = 0.0001)
# base_model.compile(optimizer = opt, loss = 'binary_crossentropy', metrics = 'accuracy')
model.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = 'accuracy')
model_extra.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = 'accuracy')

# model.summary()
# base_model.summary()

In [3]:
# directory = './ImageNet/organized_validation/'

# data_batches = tf.keras.preprocessing.image_dataset_from_directory(
#     directory,
#     labels="inferred",
#     label_mode="categorical",
# #     class_names=None,
#     color_mode="rgb",
#     batch_size=1,
#     image_size=(224, 224),
#     shuffle=True,
#     seed=13,
# #     validation_split=None,
# #     subset=None,
#     interpolation="bilinear"
# #     follow_links=False,
# )

In [4]:
directory = './ImageNet/organized_validation/'

resnet_preprocess = keras.applications.resnet.preprocess_input

val_id_gen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function = resnet_preprocess)
# val_gen = keras.preprocessing.image.ImageDataGenerator()
# val_id_gen = tensorflow.keras.preprocessing.image.ImageDataGenerator()
# val_gen = ImageDataGenerator()


val_gen = val_id_gen.flow_from_directory(
    directory, 
    target_size=(224, 224),
    color_mode='rgb', 
    classes=None,
    class_mode='categorical', 
    batch_size=1, 
    shuffle=False, 
    seed=13,
#     save_to_dir=None, 
#     save_prefix='', 
#     save_format='png', f
#     ollow_links=False,
#     subset=None, 
    interpolation='nearest'
)

# next(val_gen)

Found 50000 images belonging to 1000 classes.


In [5]:
x_val = np.zeros((30000, 224, 224, 3))
y_val = np.zeros((30000, 1000))
for i in range(30000):
    if i % 1000 == 0:
        print(i)
        
    x, y = val_gen.next()
    x_val[i, :, :, :] = x
    y_val[i, :] = y
    
x_train, x_val, y_train, y_val = train_test_split(x_val, y_val, train_size = .75, shuffle = True)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000


In [ ]:
# for i in range(400):
#     x, y = val_gen.next()
    
#     plt.clf()
#     plt.imshow(x[0, :, :, :]/255)
#     plt.show()
    
#     print(y)

# for x in x_val:
#     plt.clf()
#     plt.imshow(x/255)
#     plt.show()

In [ ]:
# Freeze all but last few layers:
# for layer in model.layers:
#     layer.trainable = False
    
# model.layers[-1].trainable = True
# model.layers[-2].trainable = True
# model.layers[-3].trainable = True
# base_model.layers[-4].trainable = True



In [6]:
history = model.fit(x_train, y_train, validation_data = (x_val, y_val), epochs = 50, verbose = 1, shuffle = True)

Epoch 1/50
704/704 [==============================] - 2016s 3s/step - loss: 3.0893 - accuracy: 0.4093 - val_loss: 1.6870 - val_accuracy: 0.5901
Epoch 2/50
704/704 [==============================] - 1893s 3s/step - loss: 1.0294 - accuracy: 0.7353 - val_loss: 1.4526 - val_accuracy: 0.6291
Epoch 3/50
704/704 [==============================] - 1884s 3s/step - loss: 0.6576 - accuracy: 0.8296 - val_loss: 1.3943 - val_accuracy: 0.6419
Epoch 4/50
704/704 [==============================] - 1939s 3s/step - loss: 0.4371 - accuracy: 0.8922 - val_loss: 1.3985 - val_accuracy: 0.6433
Epoch 5/50
704/704 [==============================] - 1938s 3s/step - loss: 0.2888 - accuracy: 0.9375 - val_loss: 1.3807 - val_accuracy: 0.6561
Epoch 6/50
704/704 [==============================] - 1910s 3s/step - loss: 0.1919 - accuracy: 0.9652 - val_loss: 1.4108 - val_accuracy: 0.6557
Epoch 7/50
704/704 [==============================] - 1897s 3s/step - loss: 0.1309 - accuracy: 0.9803 - val_loss: 1.4011 - val_accuracy:

KeyboardInterrupt: 

In [ ]:
history = model.fit(x_train, y_train, validation_data = (x_val, y_val), epochs = 10, verbose = 1, shuffle = True)

In [ ]:
result = model.evaluate(x_val, y_val, verbose = 1)

In [ ]:
# Get static dataset:

# x_vals = np.zeros((400, 224, 224, 3))
# y_vals = np.zeros((400, 8))
# for i in range(400):
#     x, y = val_gen.next()
#     x_vals[i, :, :, :] = x
#     y_vals[i, :] = y
    

In [ ]:
result = model.evaluate(x_vals, y_vals, verbose = 1)

In [7]:
out = model_extra.predict(x_val, verbose = 1)

235/235 [==============================] - 473s 2s/step


In [ ]:
print(out[1].shape)

In [ ]:
dist_from_first = []
image_dict = {}
index = 1444
first_features = out[2][index]
first_image = x_val[index][:,:,:]
print(first_image.shape)

# hash images:
for idx in range(x_val.shape[0]):
    image_dict[str(out[2][idx])] = x_val[idx] 

plt.clf()
plt.imshow(first_image/255)
plt.show()

# for i in range(400):
#     if i % 25 == 0:
#         print(i)
for idx, item in enumerate(out[2]):
    dist_from_first.append((np.linalg.norm(first_features-item), (str(item))))

dist_from_first.sort()
# print(dist_from_first)

for i in range(20):
    plt.clf()
    img = image_dict[dist_from_first[i][1]]
    plt.imshow(img/255)
    plt.show()